# Credit Risk Model Analysis

### Import Dependencies

In [11]:
# import data manipulation libraries
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split


# import metrics libraries
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced


# import classification libraries
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.ensemble import EasyEnsembleClassifier

# import sampling libraries
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import ClusterCentroids
from imblearn.combine import SMOTEENN

# import regression
from sklearn.linear_model import LogisticRegression



### Prep and Clean Data

In [12]:
columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]


# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

/var/folders/9_/j1y8d7d16j72pf4nkz7l4y9w0000gn/T/ipykernel_35561/665906299.py:31: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=1)[:-2]


,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


### Evaluate the Models

In [21]:
def learning_model_comparison (df) :

    # ignore warnings for cleaner output
    import warnings
    warnings.filterwarnings('ignore')

    # Create our features
    X = df.drop('loan_status',axis=1)
    X = pd.get_dummies(X)
    # Create our target
    y = df['loan_status'].to_frame()

    # Split the X and y into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=1)
    X_train_int = X_train.astype(int)

    # create models to test
    ROS = RandomOverSampler(random_state=1)
    smote = SMOTE(random_state=1, sampling_strategy='auto')
    cc = ClusterCentroids(random_state=1)
    smoteenn = SMOTEENN(random_state=1)
    lr = LogisticRegression(solver='lbfgs', random_state=1)
    brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
    eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
    
    # create lists of models
    sampler_names = ['RandomOverSampler', 'SMOTE', 'ClusterCentroids', 'SMOTEENN', ]
    samplers = [ROS, smote, cc, smoteenn,]
    ensemble_names = ['BalancedRandomForestClassifier', 'EasyEnsembleClassifier']
    ensembles = [brf, eec]
    
    # evaluate the sampling models
    for i, model in enumerate(samplers):
        if i == 2:
            X_resampled, y_resampled = model.fit_resample(X_train_int, y_train)
        else:
            X_resampled, y_resampled = model.fit_resample(X_train, y_train)

        lr.fit(X_resampled, y_resampled)
        sample_pred = lr.predict(X_test)

        print(sampler_names[i]+':\n')
        print('Balanced Accuracy Score: \n')
        print(balanced_accuracy_score(y_test, sample_pred))
        print('\n\n')
        print('Confusion Matrix: \n')
        print(confusion_matrix(y_test, sample_pred))
        print('\n\n')
        print('Classification Report: \n')
        print(classification_report_imbalanced(y_test, sample_pred))

    # evaluate the ensemble models
    for i, model in enumerate(ensembles):
        model.fit(X_train, y_train)
        ensemble_pred = model.predict(X_test)

        print(ensemble_names[i]+':\n\n')
        print('Balanced Accuracy Score: \n')
        print(balanced_accuracy_score(y_test, ensemble_pred))
        print('\n')
        print('Confusion Matrix: \n')
        print(confusion_matrix(y_test, ensemble_pred))
        print('\n')
        print('Classification Report: \n')
        print(classification_report_imbalanced(y_test, ensemble_pred))
        print('\n\n')



In [22]:
learning_model_comparison(df=df)

RandomOverSampler:

Balanced Accuracy Score: 

0.6640245116653853



Confusion Matrix: 

[[   72    29]
 [ 6582 10522]]



Classification Report: 

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.71      0.62      0.02      0.66      0.44       101
   low_risk       1.00      0.62      0.71      0.76      0.66      0.43     17104

avg / total       0.99      0.62      0.71      0.76      0.66      0.43     17205

SMOTE:

Balanced Accuracy Score: 

0.6556413183413758



Confusion Matrix: 

[[   64    37]
 [ 5514 11590]]



Classification Report: 

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.63      0.68      0.02      0.66      0.43       101
   low_risk       1.00      0.68      0.63      0.81      0.66      0.43     17104

avg / total       0.99      0.68      0.63      0.80      0.66      0.43     17205

ClusterCentroids:

Balanced Accuracy Score: 

